In [1]:
import pandas as pd
import numpy as np

In [19]:
data = pd.ExcelFile('../data/datos.xlsx')

## Consumo

In [8]:

df = data.parse('Consumo')  

df_cleaned = df.dropna(how='all')

tablas_config = {
    'Cocaína': {
        'start_col': 0,
        'cols': ['País', 'Cocaína', 'Categoría_Cocaína']
    },
    'Cannabis': {
        'start_col': 4,
        'cols': ['País', 'Cannabis', 'Categoría_Cannabis']
    },
    'Metanfetaminas': {
        'start_col': 8,
        'cols': ['País', 'Metanfetaminas', 'Categoría_Metanfetaminas']
    },
    'Opioides': {
        'start_col': 12,
        'cols': ['País', 'Opioides', 'Categoría_Opioides']
    }
}

def procesar_tabla(df, start_col, nombres_columnas):
    """Procesa cada tabla individual respetando tu estructura real"""
    try:
        tabla = df.iloc[2:, start_col:start_col+3].copy()
        tabla.columns = nombres_columnas
        
        tabla = tabla.dropna(subset=['País'])
        col_valor = nombres_columnas[1]
        tabla[col_valor] = pd.to_numeric(tabla[col_valor], errors='coerce')
        
        return tabla[['País', col_valor]]
    
    except Exception as e:
        print(f"Error procesando {nombres_columnas[1]}: {str(e)}")
        return pd.DataFrame()

dataframes = {}
for droga, config in tablas_config.items():
    temp_df = procesar_tabla(df_cleaned, 
                           config['start_col'], 
                           config['cols'])
    dataframes[droga] = temp_df

df_final = dataframes['Cocaína']
for droga in ['Cannabis', 'Metanfetaminas', 'Opioides']:
    df_final = df_final.merge(dataframes[droga], 
                            on='País', 
                            how='outer')

def aplicar_categorias(df):
    categorizadores = {
        'Cocaína': [
            (1.2, 'Alta'),
            (0.6, 'Media'),
            (0.29, 'Baja'),
            (-np.inf, 'Muy baja')
        ],
        'Cannabis': [
            (6.8, 'Alta'),
            (3.9, 'Media'),
            (2.3, 'Baja'),
            (-np.inf, 'Muy baja')
        ],
        'Metanfetaminas': [
            (0.9, 'Alta'),
            (0.48, 'Media'),
            (0.17, 'Baja'),
            (-np.inf, 'Muy Baja')
        ],
        'Opioides': [
            (1, 'Alta'),
            (0.5, 'Media'),
            (0.26, 'Baja'),
            (-np.inf, 'Muy baja')
        ]
    }
    
    for droga, limites in categorizadores.items():
        col_categoria = f'Categoría_{droga}'
        df[col_categoria] = df[droga].apply(
            lambda x: next((cat for lim, cat in limites if x > lim), 'Sin dato')
        )
    
    return df

df_final = aplicar_categorias(df_final)

column_order = ['País'] + \
    [f'{d}|Categoría_{d}' for d in tablas_config.keys()]
    
df_final = df_final.fillna(0).sort_values('País').reset_index(drop=True)

In [9]:
df_final.to_csv('../data/consumo.csv', index=False)

## Produccion

In [15]:
import pandas as pd

# Datos crudos (copiar exactamente los datos que proporcionaste)
cocaina_raw = [
    ('Colombia', 24514, 'ALTA'), ('Bolivia', 3446, None), ('Peru', 2202, None),
    # ... (copiar todos los registros de cocaína que mostraste)
]

opioides_raw = [
    ('Malasia', 40, 'ALTA'), ('Grecia', 22, None), ('República Checa', 19, None),
    # ... (copiar todos los registros de opioides que mostraste)
]

metanfetaminas_raw = [
    ('Estados Unidos', 2806, 'ALTA'), ('República Checa', 1021, None), ('China', 479, None),
    # ... (copiar todos los registros de metanfetaminas que mostraste)
]

# Función para crear DataFrame base
def crear_dataframe():
    paises = set([p[0] for p in cocaina_raw + opioides_raw + metanfetaminas_raw])
    data = []
    
    for pais in paises:
        row = {'País': pais}
        
        # Buscar en cada lista
        for lista, sufijo in zip([cocaina_raw, opioides_raw, metanfetaminas_raw], ['_coca', '_opio', '_meta']):
            match = next((x for x in lista if x[0] == pais), (None, None, None))
            row[f'laboratorio{sufijo}'] = match[1]
        
        data.append(row)
    
    return pd.DataFrame(data).replace({None: pd.NA})

df = crear_dataframe()

# Funciones de categorización (las que proporcionaste)
def asignar_categoria_coca(x):
    if x > 70: return "ALTA"
    elif x > 12: return "MEDIA"
    elif x > 1: return "BAJA"
    else: return "MUY BAJA"

def asignar_categoria_opio(x):
    if x > 18: return "ALTA"
    elif x > 8: return "MEDIA"
    elif x > 2: return "BAJA"
    else: return "MUY BAJA"

def asignar_categoria_meta(x):
    if x > 37: return "ALTA"
    elif x > 8: return "MEDIA"
    elif x > 2: return "BAJA"
    else: return "MUY BAJA"

# Aplicar categorización automática
def aplicar_categorias(df):
    for droga, func in zip(['coca', 'opio', 'meta'], 
                          [asignar_categoria_coca, asignar_categoria_opio, asignar_categoria_meta]):
        col_lab = f'laboratorio_{droga}'
        col_cat = f'cat_{droga}'
        
        df[col_cat] = df[col_lab].apply(
            lambda x: func(x).title() if pd.notna(x) else 'Sin dato'
        )
    return df

# Resultado final
df_final = aplicar_categorias(df)[[
    'País', 
    'laboratorio_coca', 'cat_coca',
    'laboratorio_opio', 'cat_opio',
    'laboratorio_meta', 'cat_meta'
]].sort_values('País').reset_index(drop=True)

In [17]:
df_final.to_csv('../data/produccion.csv', index=False)

## Trafico

In [20]:
df_trafico = data.parse('Tráfico', header=None)

df_trafico_clean = df_trafico.dropna(how='all')

header_row = 1  # La fila REAL con los encabezados es la 1 (índice 1)
df_trafico_clean.columns = ['Col_extra', 'País', 'Cocaína', 'Cannabis', 'Metanfetaminas', 'Heroína']

df_trafico_clean = df_trafico_clean.drop(columns=['Col_extra'])

df_trafico_final = df_trafico_clean.iloc[2:].copy()

df_trafico_final = df_trafico_final.reset_index(drop=True)
df_trafico_final['País'] = df_trafico_final['País'].str.strip()

def limpiar_valores(valor):
    if isinstance(valor, str):
        try:
            return float(valor.replace('$', '').replace(',', '').strip())
        except:
            return np.nan
    return valor

for col in ['Cocaína', 'Cannabis', 'Metanfetaminas', 'Heroína']:
    df_trafico_final[col] = df_trafico_final[col].apply(limpiar_valores)

In [21]:
df_trafico_final.head(10)

,País,Cocaína,Cannabis,Metanfetaminas,Heroína
0,Sri Lanka,177609.980000,1731.53,4567.113736,145917.640000
1,Australia,159530.390000,NaN,85635.359116,142964.640000
2,Finlandia,94686.555524,5751.15,14874.141876,53374.333333
3,Palestina,75000.000000,4464.29,NaN,NaN
4,Moldavia,71174.380000,NaN,NaN,126448.890000
5,Estonia,71174.380000,7278.84,4744.960000,NaN
6,Egipto,63836.580000,2168.57,31709.791984,6341.960000
7,India,59666.680000,753.22,9763.638583,191331.410000
8,Austria,59471.364769,NaN,47449.580000,37402.217251
9,Reino Unido,57087.631986,2562.87,44987.146530,33375.054519


In [22]:
df_trafico_final.to_csv('../data/trafico.csv', index=False)

## Mercado

In [23]:
df_mercado = data.parse('Estimación de Mercado', header=None)
df_mercado_clean = df_mercado.dropna(how='all', axis=0)


def procesar_columna_ajustada(df, start_col):
    tabla = df.iloc[1:, start_col:start_col+2].copy()  
    tabla.columns = ['País', df.iloc[3, start_col+1]]  
    tabla = tabla.dropna(subset=['País'])
    return tabla.reset_index(drop=True)


poblacion = procesar_columna_ajustada(df_mercado_clean, 1).rename(
    columns={df_mercado_clean.iloc[3,2]: 'Población'})

consumo = procesar_columna_ajustada(df_mercado_clean, 4).rename(
    columns={df_mercado_clean.iloc[3,5]: 'Consumo_Tn'})

mercado = procesar_columna_ajustada(df_mercado_clean, 7).rename(
    columns={df_mercado_clean.iloc[3,8]: 'Mercado_USD'})

def limpiar_final(df):
    df['País'] = df['País'].astype(str).str.strip()
    df = df.replace({
        'Reino Unido (Irlanda del Norte)': 'Reino Unido',
        'Chequia': 'República Checa',
        'México': 'México' 
    })
    if 'Mercado_USD' in df.columns:
        df['Mercado_USD'] = df['Mercado_USD'].astype(str).str.replace('[$,]', '', regex=True).replace('nan', np.nan).astype(float)
    return df

df_final = (
    limpiar_final(poblacion)
    .merge(limpiar_final(consumo), on='País', how='outer')
    .merge(limpiar_final(mercado), on='País', how='outer')
    .sort_values('Población', ascending=False)
    .reset_index(drop=True)
)

df_final[['Población', 'Consumo_Tn']] = df_final[['Población', 'Consumo_Tn']].apply(pd.to_numeric, errors='coerce')

In [24]:
df_final.head(10)

,País,Población,Consumo_Tn,Mercado_USD
0,Estados Unidos,7409071,614.952893,1.875606e+10
1,Brasil,4488953,372.583099,9.314577e+08
2,India,1300000,107.900000,6.438035e+09
3,España,1056208,87.665264,3.748080e+09
4,Alemania,831375,69.004125,3.039619e+09
5,Francia,795088,65.992304,2.544187e+09
6,México,784635,65.124705,6.643807e+08
7,Australia,668847,55.514301,8.856218e+09
8,Italia,656593,54.497219,2.396256e+09
9,Argentina,506004,41.998332,2.099917e+08


In [25]:
df_final.to_csv('../data/mercado.csv', index=False)